# Reading GISAID CoV variant reporting datafile (2023/02/09)
This Jupyter script reads and processes the COVID-19 variants datafile. The datafile is presented in an Excel file, with the following information:
1. VOC/VOI are separated into different Excel sheets
2. Each sheet has all the countries that have reported any genomic observation(s)
3. Rows are countries, with two subrows for 'count' and 'total'. 'total' refers to the total observations for all COVID variants for that country and week, 'count' refers to only that particular variant. The 'total' cell will be empty if 'count' is also empty.
4. Columns are week of detection, in yyyy-mm-dd format
5. Unreported cells = not detected for that country and week.

This script does the following:
1. Extract information for only the Omicron variant
2. Section the dataset at the start of Omicron detection, till a certain time (default: 4 months) after that point
3. Eliminate countries that do not have much reporting in the time period.
4. Adapt the data to be easier to process in code, at the cost of human readability.
5. Save the processed data as a separate data file(s). By default, two copies are saved:
    - a comma-separated text file (.csv). The CSV is for basic visual reference, as the pickle file is not in a human-readable format
    - a pickled Pandas dataframe (.pickletable). The pickle is for further data processing, as it is faster to load and requires less processing

This script is written in the Python programming language.

In [4]:
import os, sys

os.chdir('E:\Downloads\@Books\@TPM\@PDPC\@data_analysis\model_build')
sys.path.append('E:\Downloads\@Books\@TPM\@PDPC\@data_analysis\model_build')
import pandas as pd
# import openpyxl

In [5]:
target_dir = '../data_gisaid'
target_file = 'gisaid_variants_statistics_2023_02_09_1433.xlsx'

# define sheet to read
read_sheets = "VOC Omicron"

base_read = pd.read_excel(io=f'{target_dir}/{target_file}',
                          sheet_name=read_sheets,
                          engine='openpyxl',
                          parse_dates=True)
# rename some columns to intuitive names, hardcoded
base_read.rename(columns= {'Unnamed: 0': 'Country',
                  'VOC Omicron GRA (B.1.1.529+BA.*) ': 'Type'},
                 inplace=True)

# --- Data Pre-cleaning Operations ---
# 1a: copying names to index TODO document in Markdown
c_index = base_read['Country']
if not c_index.notna().all(): # only process if DF is raw
    country_names = c_index[c_index.notna()]
    country_names.index += 1
    base_read.loc[country_names.index,'Country'] = country_names
    del country_names  # not used anymore

# 1b: convert to multiindex
base_read.set_index(['Country','Type'], inplace=True)
# 2: section for time period of interest
# derive first time of incidence, and add a delta of N from that point onwards (eg. we know each column yields a week
# eliminate all columns before?
col_first = None
col_first_idx = None
col_delta = 12 * 4  # in weeks, 1 year
for idx, col in enumerate(base_read.columns):
    if base_read[col].notna().any():
        col_first_idx = idx
        col_first = col
        break
if col_first is None: # sanity check
    raise ValueError("Cannot find any date column with first reporting of variant")
base_read = base_read.iloc[:, slice(col_first_idx, col_first_idx+col_delta)]

# 3: section for countries with entries
drop_thresh = col_delta - 2
base_read.dropna(thresh=drop_thresh, inplace=True)
# 4: convert to nullable int dtype
base_read = base_read.astype(pd.Int32Dtype())
# TODO ask R if she knows the genome submission rate is affected by novelty or standard sampling rate
# LAST: save to alternate file
# save_name = "gisaid_omicron_2111_processed"
# base_read.to_pickle(f'{target_dir}/{save_name}.pickletable')
# base_read.to_csv(f'{target_dir}/{save_name}_(preview).csv', sep=',')

In [6]:
variants_dir = "../data_gisaid"
# variants_file = "gisaid_omicron_2111_processed.pickletable"
# variants_df = pd.read_pickle(f'{variants_dir}/{variants_file}')
# clean up data for specifically SA
sa_obs_df = base_read.loc[("South Africa",), :].T
sa_obs_df['ratio'] = sa_obs_df['count'] / sa_obs_df['total']

# save
save_file = "omicron_samples_SA_long"
sa_obs_df.to_pickle(f"{variants_dir}/{save_file}.pickletable")
sa_obs_df.to_csv(f'{variants_dir}/{save_file}.csv', sep=',')

In [ ]:
# Test to see if the file is read back in the exact same manner
# base_read3 = pd.read_pickle(f'{target_dir}/{save_name}')

